In [15]:
#pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials as cc
import pandas as pd
import json
import pandas as pd
import os
from IPython.display import display
# d = json.load(open('data/mpd.slice.0-999.json'))
import warnings
# from playlists extract tracks with corresponding features from spotipy
import time
import numpy as np
import webbrowser
from pandas._libs.internals import defaultdict
import random
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# create list of playlists
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
display(playlists.head())

C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\470225530.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='data\\mpd.slice.0-999.json' mode='r' encoding='cp1252'>
  d = json.load(open(f))
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\470225530.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='data\\mpd.slice.1000-1999.json' mode='r' encoding='cp1252'>
  d = json.load(open(f))
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\470225530.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='data\\mpd.slice.10000-10999.json' mode='r' encoding='cp1252'>
  d = json.load(open(f))
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\470225530.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='data\\mpd.slice.100000-100999.json' mode='r' encoding='cp1252'>
  d = json.load(open(f))
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\470225530.py:11: ResourceWarning: unclosed file <_io.TextIOWrapper name='data\\mpd.slice.101000-101999.json' mode='r

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN


In [17]:
# api setup
client_id = '898dd71dc932407e85921f0ac79f0127'
client_secret = '16d994f070064371beb8758f32d64180'
#https://developer.spotify.com/dashboard/applications/898dd71dc932407e85921f0ac79f0127
client_cc = cc(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_cc)

In [18]:
# audio feature extraction
sleep_min = 1
sleep_max = 2
start_time = time.time()
request_count = 0
tracks_w_features = pd.DataFrame()
recommendations = pd.DataFrame()
playlists_no = 5
songs_max = 100

for i in range(0,playlists_no):
    request_count+=1
    if request_count % 53 == 0:
        time.sleep(np.random.uniform(sleep_min, sleep_max))
    # extract audio features using spotipy api (called sp with credentials)
    if len(playlists['tracks'][i]) < songs_max:
        tracks = pd.DataFrame(sp.audio_features(pd.DataFrame(playlists['tracks'][i]).reset_index(drop=True).track_uri))
        tracks_w_features = tracks_w_features.append(tracks)
display(tracks_w_features.head())

C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\1151244345.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_features.append(tracks)
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\1151244345.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_features.append(tracks)
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\1151244345.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_features.append(tracks)
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_27008\1151244345.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4


In [19]:
# remove useless-categorical columns, can use index to get song information later
input = tracks_w_features.drop(columns=['time_signature', 'duration_ms','analysis_url','track_href','uri','id','type', 'mode', 'key', 'tempo']).copy()
display(input.iloc[0])
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='once')

danceability        0.90400
energy              0.81300
loudness           -7.10500
speechiness         0.12100
acousticness        0.03110
instrumentalness    0.00697
liveness            0.04710
valence             0.81000
Name: 0, dtype: float64

In [20]:
# remove duplicates
tracks_w_features.drop_duplicates()
display(tracks_w_features)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,0.577,0.497,1,-9.371,0,0.0235,0.06190,0.000011,0.1100,0.279,99.662,audio_features,3rfDfPjgHDQ1BUaSHr9xoI,spotify:track:3rfDfPjgHDQ1BUaSHr9xoI,https://api.spotify.com/v1/tracks/3rfDfPjgHDQ1...,https://api.spotify.com/v1/audio-analysis/3rfD...,269467,4
13,0.509,0.537,5,-6.834,1,0.0384,0.34700,0.000007,0.0805,0.311,170.030,audio_features,6si71supnBUhddjR2FJc2L,spotify:track:6si71supnBUhddjR2FJc2L,https://api.spotify.com/v1/tracks/6si71supnBUh...,https://api.spotify.com/v1/audio-analysis/6si7...,240040,4
14,0.734,0.580,9,-6.619,0,0.0321,0.23800,0.000600,0.0298,0.962,119.336,audio_features,3XEtw7t4V5sfd2vtkp0ql7,spotify:track:3XEtw7t4V5sfd2vtkp0ql7,https://api.spotify.com/v1/tracks/3XEtw7t4V5sf...,https://api.spotify.com/v1/audio-analysis/3XEt...,258693,4
15,0.863,0.452,6,-10.839,0,0.0678,0.03760,0.021500,0.0532,0.544,99.102,audio_features,6HS5QOvhd4w3XqJMDaEWcv,spotify:track:6HS5QOvhd4w3XqJMDaEWcv,https://api.spotify.com/v1/tracks/6HS5QOvhd4w3...,https://api.spotify.com/v1/audio-analysis/6HS5...,227867,4


In [21]:
# cos similarity
df = pd.DataFrame(cosine_similarity(input, dense_output=True))

In [22]:
# feature space creation

# 3 by input size matrix
# columns as follows:
# song a, song b, songs a similarity with b
# song a, song c, song a similarity with c
# song a, song d, song a similarity with d
feature_space = []
for i,j in enumerate(df.values):
    for p in range(len(j)):
        feature_space.append([i,p,j[p]])

In [23]:
# remove 1's in diagonals of cos sim matrix
feature_space_refined = []
for i in range(len(feature_space)):
    if feature_space[i][0]==feature_space[i][1]:
        feature_space_refined.append([feature_space[i][0],feature_space[i][1],0])
    else:
        feature_space_refined.append(feature_space[i])
display(feature_space_refined[0:5])
display(feature_space_refined[len(feature_space_refined)-5:len(feature_space_refined)])

[[0, 0, 0],
 [0, 1, 0.985990401450441],
 [0, 2, 0.999525826013183],
 [0, 3, 0.9991953986710053],
 [0, 4, 0.9961907611354008]]

[[171, 167, 0.993633293897211],
 [171, 168, 0.9961925527861686],
 [171, 169, 0.9959611187117934],
 [171, 170, 0.9935600212772345],
 [171, 171, 0]]

In [24]:
# add most similar songs
dict = defaultdict(list)
for i in range(len(feature_space_refined)):
    dict[feature_space_refined[i][0]].append(feature_space_refined[i][2])
df_updated = pd.DataFrame(dict)
display(df_updated)

# find max similarity and sort using argmax
sorted = []
for i in range(len(df_updated)):
    sorted.append(np.argmax(df_updated[i]))
#display(sorted)

# get the highest similarity of index position
sorted_updated = []
for i in sorted:
    sorted_updated.append(sorted[i])

similar_songs = pd.DataFrame (sorted_updated, columns=['Similar Song'])

similarity_value = pd.DataFrame(round(df_updated.max(axis=1),12), columns=['Similarity Value'])

parsed_songs = pd.DataFrame(sorted, columns=['Parsed Song'])

cs_df = pd.concat([parsed_songs,similar_songs,similarity_value],axis=1)

display(cs_df.head(5))
display(cs_df.tail(5))

,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
0,0.000000,0.985990,0.999526,0.999195,0.996191,0.996751,0.996798,0.994150,0.988217,0.917779,...,0.995260,0.997120,0.982241,0.994231,0.997556,0.992624,0.994704,0.998505,0.994398,0.997939
1,0.985990,0.000000,0.983436,0.989170,0.993267,0.992748,0.971727,0.987949,0.997058,0.969643,...,0.970347,0.973534,0.940973,0.979820,0.973811,0.960024,0.967053,0.984379,0.963842,0.982054
2,0.999526,0.983436,0.000000,0.998071,0.994211,0.995019,0.997524,0.994453,0.985394,0.911937,...,0.995565,0.997437,0.984478,0.995711,0.998479,0.994126,0.995663,0.997894,0.995334,0.998597
3,0.999195,0.989170,0.998071,0.000000,0.997353,0.997572,0.995074,0.994549,0.992771,0.926988,...,0.994227,0.994641,0.978786,0.991828,0.994764,0.988746,0.992705,0.998724,0.991152,0.996252
4,0.996191,0.993267,0.994211,0.997353,0.000000,0.997364,0.988980,0.994669,0.994219,0.937339,...,0.988798,0.990794,0.969157,0.990848,0.989933,0.982494,0.986413,0.994345,0.985171,0.992550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,0.992624,0.960024,0.994126,0.988746,0.982494,0.981417,0.998068,0.983944,0.965231,0.865235,...,0.996910,0.997483,0.996408,0.991698,0.998371,0.000000,0.998514,0.991409,0.999533,0.993633
168,0.994704,0.967053,0.995663,0.992705,0.986413,0.985765,0.998940,0.989155,0.974369,0.881001,...,0.999577,0.998281,0.995517,0.993468,0.998435,0.998514,0.000000,0.994422,0.998180,0.996193
169,0.998505,0.984379,0.997894,0.998724,0.994345,0.993130,0.997288,0.990618,0.988373,0.915846,...,0.994955,0.994226,0.983964,0.991870,0.996180,0.991409,0.994422,0.000000,0.993689,0.995961
170,0.994398,0.963842,0.995334,0.991152,0.985171,0.983926,0.998899,0.984191,0.968756,0.871371,...,0.996912,0.997447,0.995288,0.990761,0.998593,0.999533,0.998180,0.993689,0.000000,0.993560


,Parsed Song,Similar Song,Similarity Value
0,2,0,0.999526
1,140,1,0.999529
2,0,2,0.999526
3,15,135,0.999642
4,135,20,0.998580


,Parsed Song,Similar Song,Similarity Value
167,89,66,0.999784
168,70,168,0.999772
169,158,68,0.998942
170,167,89,0.999533
171,84,87,0.999447


In [25]:
# get most similar songs
random_pair = random.randint(0, similar_songs.shape[0])
print('first comparison song index', random_pair)

track0 = tracks_w_features.iloc[int(cs_df.iloc[random_pair][0])]
track1 = tracks_w_features.iloc[int(cs_df.iloc[random_pair][1])]

track0 = track0.drop(columns=['time_signature', 'duration_ms','analysis_url','track_href','uri','id','type']).copy()
track1 = track1.drop(columns=['time_signature', 'duration_ms','analysis_url','track_href','uri','id','type']).copy()

track0_uri = sp.track(tracks_w_features.iloc[int(cs_df.iloc[random_pair][0])].uri)
track1_uri = sp.track(tracks_w_features.iloc[int(cs_df.iloc[random_pair][1])].uri)

table = {'Track 0': track0, 'Track 1': track1}
table = pd.DataFrame(table)
display(table)

track0_url = track0_uri['external_urls']['spotify']
track1_url = track1_uri['external_urls']['spotify']

first comparison song index 86


,Track 0,Track 1
danceability,0.277,0.363
energy,0.67,0.554
key,6,2
loudness,-9.381,-10.784
mode,1,0
speechiness,0.0376,0.0317
acousticness,0.000063,0.0885
instrumentalness,0.751,0.973
liveness,0.237,0.261
valence,0.217,0.408


In [26]:
# open in browser
webbrowser.open(track0_url)
webbrowser.open(track1_url)

True